In [8]:
import plotly
plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')

In [10]:
import plotly
plotly.tools.set_config_file(world_readable=False,
                             sharing='private')

In [11]:
import plotly
plotly.tools.set_config_file(plotly_domain='https://plotly.your-company.com',
                             plotly_streaming_domain='https://stream-plotly.your-company.com')

In [12]:

import plotly
plotly.tools.set_config_file(plotly_domain='https://plotly.your-company.com',
                             plotly_streaming_domain='https://stream-plotly.your-company.com',
                             world_readable=False,
                             sharing='private')

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *

trace0 = Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = Data([trace0, trace1])

py.plot(data, filename = 'basic-line')

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *

trace0 = Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = Data([trace0, trace1])

py.iplot(data, filename = 'basic-line')